In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-
import os
import gc

import numpy as np
import torch
import torch.backends.cudnn as cudnn
from torch import nn

from dataloader import mnist
from models import ResNet18
from src import freeze_influence, hessians, selection

device = "cuda" if torch.cuda.is_available() else "cpu"
target_removal_label = 0

In [3]:
def load_net(net, path):
    assert os.path.isfile(path), "Error: no checkpoint file found!"
    checkpoint = torch.load(path)
    net.load_state_dict(checkpoint["net"])
    return net


def save_net(net, path):
    dir, filename = os.path.split(path)
    if not os.path.isdir(dir):
        os.makedirs(dir)

    state = {
        "net": net.state_dict(),
    }
    torch.save(state, path)


def test(net, dataloader, criterion, label, include):
    net.eval()
    with torch.no_grad():
        net_loss = 0
        correct = 0
        total = 0
        num_data = 0
        for _, (inputs, targets) in enumerate(dataloader):
            if include:
                idx = targets == label
            else:
                idx = targets != label
            inputs = inputs[idx]
            targets = targets[idx]
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            net_loss += loss * len(inputs)
            num_data +=  len(inputs)

            total += targets.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()

        accuracy = correct / total * 100
        net_loss /= num_data
        return net_loss, accuracy

def influence_test(net, dataloader, criterion, target_label):
    def sample_test(net, criterion, inputs, targets):
        net.eval()
        with torch.no_grad():
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            _, predicted = outputs.max(1)
            correct = predicted.eq(targets).sum().item()

            return loss, correct

    self_loss = 0
    self_correct = 0
    num_self_inputs = 0
    
    exclusive_loss = 0
    exclusive_correct = 0
    num_exclusive_inputs = 0
    
    for _, (inputs, targets) in enumerate(dataloader):
        target_idx = (targets == target_label)
        batch_self_loss, batch_self_correct = sample_test(net, criterion, inputs[target_idx], targets[target_idx])
        batch_exclusive_loss, batch_exclusive_correct = sample_test(net, criterion, inputs[~target_idx], targets[~target_idx])
        
        len_self_batch = len(inputs[target_idx])
        self_loss += batch_self_loss * len_self_batch
        self_correct += batch_self_correct
        num_self_inputs += len_self_batch
        
        len_exclusive_batch = len(inputs[~target_idx])
        exclusive_loss += batch_exclusive_loss * len_exclusive_batch
        exclusive_correct += batch_exclusive_correct
        num_exclusive_inputs += len_exclusive_batch
        
    self_loss /= num_self_inputs
    self_acc = self_correct / num_self_inputs * 100
    exclusive_loss /= num_exclusive_inputs
    exclusive_acc = exclusive_correct / num_exclusive_inputs * 100
    
    return self_loss, self_acc, exclusive_loss, exclusive_acc

def projected_influence(net, total_loss, target_loss, index_list, tol, step, max_iter, verbose):
    num_param = sum(p.numel() for p in net.parameters() if p.requires_grad)
    full_param_index_list = np.arange(num_param)
    influence = hessians.generalized_influence(
        net, total_loss, target_loss, full_param_index_list, tol=tol, step=step, max_iter=max_iter, verbose=verbose
    )
    return influence[idx]

def f1_score(self_acc, test_acc):
    self_acc /= 100
    test_acc /= 100
    if self_acc == 1 and test_acc == 0:
        return 0
    return 2 * (1 - self_acc) * test_acc / (1 - self_acc + test_acc)

In [4]:
net = ResNet18(1).to(device)
net_name = "ResNet18"

if device == "cuda":
    cudnn.benchmark = True

net_path = f"checkpoints/tab2/{net_name}/cross_entropy/ckpt_0.0.pth"
net = load_net(net, net_path)

net.eval()
num_param = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(
    f"==> Building {net_name} finished. "
    + f"\n    Number of parameters: {num_param}"
)

criterion = nn.CrossEntropyLoss()

# Data
print("==> Preparing data..")
batch_size = 512
num_workers = 16
num_sample_batch = 1
num_target_sample = 512

data_loader = mnist.MNISTDataLoader(batch_size, num_workers, validation=False)
train_loader, test_loader = data_loader.get_data_loaders()

loss, acc = test(net, test_loader, criterion, 11, False)
print(
    f"Original loss and acc : {loss:.4f}, {acc:.2f}%"
)

==> Building ResNet18 finished. 
    Number of parameters: 11172810
==> Preparing data..
Original loss and acc : 0.0303, 99.09%


In [5]:
verbose = True
num_exp = 10

removal_inputs = list()
removal_targets = list()
for batch_idx, (inputs, targets) in enumerate(test_loader):
    idx = targets == target_removal_label
    removal_inputs.append(inputs[idx])
    removal_targets.append(targets[idx])
removal_inputs = torch.cat(removal_inputs)
removal_targets = torch.cat(removal_targets)

ratio_list = [.1, .3, .5]
result_list_TopNActivations = []
result_list_TopNGradients = []
result_list_Random = []
result_list_Threshold = []
tol = 1e-9

parser_list = [selection.TopNActivations,
              selection.TopNGradients,
              selection.Random,
              selection.Threshold]

In [6]:
for i in range(num_exp):
    sample_idx = np.random.choice(len(removal_inputs), num_target_sample, replace=False)
    for param_ratio in ratio_list:
        for parser in parser_list:

            print(f"{parser.__name__} - ratio: {param_ratio*100}%")
            # Initialize network
            net = load_net(net, net_path)

            # Compute total loss
            total_loss = 0
            for batch_idx, (inputs, targets) in enumerate(test_loader):
                if batch_idx >= num_sample_batch:
                    break
                idx = targets != target_removal_label
                inputs, targets = inputs[idx], targets[idx]
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                total_loss += criterion(outputs, targets)

            # Sampling the target removal data
            sample_removal_inputs = removal_inputs[sample_idx]
            sample_removal_targets = removal_targets[sample_idx]
            
            # Make hooks
            net_parser = parser(net, param_ratio)
            net_parser.register_hooks()

            if isinstance(net_parser, selection.TopNGradients):
                # Compute target loss
                target_loss = (
                    criterion(net(sample_removal_inputs.to(device)), sample_removal_targets.to(device))
                    * len(removal_inputs)
                    / (len(train_loader.dataset) - len(removal_inputs))
                )
                target_loss.backward()
                net_parser.remove_hooks()

            target_loss = (
                criterion(net(sample_removal_inputs.to(device)), sample_removal_targets.to(device))
                * len(removal_inputs)
                / (len(train_loader.dataset) - len(removal_inputs))
            )
            
            # Delete hooks
            index_list = net_parser.get_parameters()
            net_parser.remove_hooks()
            
            influence = hessians.generalized_influence(
                net, total_loss, target_loss, index_list, tol=tol, step=3, max_iter=30, verbose=False
            )
            del target_loss, total_loss
            gc.collect()
            torch.cuda.empty_cache()
            
            influence = influence * 0.05 / torch.norm(influence)
                
            scale = 1
            score = 0
            best_score = -1
            count = 1
            save_path = (
                f"checkpoints/tab1/{net_name}/{net_parser.__class__.__name__}/{param_ratio}_{i}.pth"
            )
            while True:
                if score < .85:
                    net_parser.update_network(influence * scale)
                else:
                    net_parser.update_network(influence * scale / 3)

#                 self_loss, self_acc = test(net, test_loader, criterion, target_removal_label, True)
#                 exclusive_loss, exclusive_acc = test(net, test_loader, criterion, target_removal_label, False)
                self_loss, self_acc, exclusive_loss, exclusive_acc = influence_test(net, test_loader, 
                                                                                    criterion, target_removal_label)
                score = f1_score(self_acc, exclusive_acc)
                
                if best_score < score:
                    best_result = [exclusive_acc, exclusive_loss, self_acc, self_loss, score]
                    best_score = score
                    save_net(net, save_path)
                    
                if verbose:
                    print(
                    f"{count} - test acc: {exclusive_acc:2.2f}, test loss: {exclusive_loss:.4f} |" +
                    f" self-acc: {self_acc:2.2f}%, self loss: {self_loss:.4f} | score: {score:.7f}",
                    end='\r'
                    )
                
                if exclusive_acc < 80 or self_acc < 0.01 or count >= 200:
                    if i == 0:
                        result_list_TopNActivations += best_result
                    elif i == 1:
                        result_list_TopNGradients += best_result
                    elif i == 2:
                        result_list_Random += best_result
                    else:
                        result_list_Threshold += best_result
                    
                    print(f"test acc: {best_result[0]:2.2f}, test loss: {best_result[1]:.4f} | " +
                          f"self-acc: {best_result[2]:2.2f}%, self loss: {best_result[3]:.4f} | " +
                          f"Score: {best_result[4]:.7f}" + " " * 20) 
                    break
                elif count >= 20 and best_score < 0.2:
                    scale *= 5
                elif count >= 50 and best_score < 0.5:
                    print(f"test acc: {best_result[0]:2.2f}, test loss: {best_result[1]:.4f} | " +
                          f"self-acc: {best_result[2]:2.2f}%, self loss: {best_result[3]:.4f} | " +
                          f"Score: {best_result[4]:.7f}" + " " * 20) 
                    break

                count += 1
                
        print("")

TopNActivations - ratio: 10.0%
test acc: 97.94, test loss: 0.0697 | self-acc: 0.00%, self loss: 3.5648 | Score: 0.9895822                    
TopNGradients - ratio: 10.0%
test acc: 98.54, test loss: 0.0493 | self-acc: 0.00%, self loss: 3.8372 | Score: 0.9926290                    
Random - ratio: 10.0%
test acc: 93.29, test loss: 0.2194 | self-acc: 0.71%, self loss: 4.3436 | Score: 0.9619595                    
Threshold - ratio: 10.0%
test acc: 94.79, test loss: 0.1868 | self-acc: 0.20%, self loss: 4.2027 | Score: 0.9722823                    

TopNActivations - ratio: 30.0%
test acc: 93.82, test loss: 0.2264 | self-acc: 0.82%, self loss: 4.1612 | Score: 0.9642986                    
TopNGradients - ratio: 30.0%
test acc: 97.37, test loss: 0.0920 | self-acc: 0.20%, self loss: 4.6977 | Score: 0.9856932                    
Random - ratio: 30.0%
test acc: 92.11, test loss: 0.2850 | self-acc: 2.14%, self loss: 3.6935 | Score: 0.9489474                    
Threshold - ratio: 30.0%
test acc

test acc: 71.94, test loss: 1.2057 | self-acc: 1.33%, self loss: 6.1965 | Score: 0.8321239                    

TopNActivations - ratio: 10.0%
test acc: 95.03, test loss: 0.1868 | self-acc: 0.00%, self loss: 3.6806 | Score: 0.9745339                    
TopNGradients - ratio: 10.0%
test acc: 98.12, test loss: 0.0674 | self-acc: 0.00%, self loss: 3.9974 | Score: 0.9904868                    
Random - ratio: 10.0%
test acc: 88.05, test loss: 0.4621 | self-acc: 1.53%, self loss: 5.0408 | Score: 0.9296799                    
Threshold - ratio: 10.0%
test acc: 84.82, test loss: 0.5502 | self-acc: 6.33%, self loss: 3.9136 | Score: 0.8902861                    

TopNActivations - ratio: 30.0%
test acc: 92.83, test loss: 0.2574 | self-acc: 0.20%, self loss: 4.4309 | Score: 0.9618542                    
TopNGradients - ratio: 30.0%
test acc: 96.40, test loss: 0.1290 | self-acc: 0.10%, self loss: 4.7946 | Score: 0.9811621                    
Random - ratio: 30.0%
test acc: 85.23, test loss: 0.59

test acc: 89.18, test loss: 0.3919 | self-acc: 0.71%, self loss: 4.2386 | Score: 0.9396170                    
Threshold - ratio: 50.0%
test acc: 90.11, test loss: 0.3718 | self-acc: 2.45%, self loss: 4.7046 | Score: 0.9368345                    



In [7]:
for param_ratio in ratio_list:
    print("")
    for parser in parser_list:
        print(f"{parser.__name__} - ratio: {param_ratio*100}%")
        
        self_loss_list = np.empty(0)
        self_acc_list = np.empty(0)
        exclusive_loss_list = np.empty(0)
        exclusive_acc_list = np.empty(0)
        
        for i in range(num_exp):

            load_path = (
                f"checkpoints/tab1/{net_name}/{parser.__name__}/{param_ratio}_{i}.pth"
            )
            net = ResNet18(1).to(device)
            net = load_net(net, load_path)
            self_loss, self_acc, exclusive_loss, exclusive_acc = influence_test(net, test_loader, 
                                                                    criterion, target_removal_label)
            
            self_loss_list = np.append(self_loss_list, self_loss.detach().cpu().numpy())
            self_acc_list = np.append(self_acc_list, self_acc)
            exclusive_loss_list = np.append(exclusive_loss_list, exclusive_loss.detach().cpu().numpy())
            exclusive_acc_list = np.append(exclusive_acc_list, exclusive_acc)
            print(
            f"  test acc: {exclusive_acc:2.2f}, test loss: {exclusive_loss:.4f} | " +
            f"self-acc: {self_acc:2.2f}%, self loss: {self_loss:.4f}",
            end='\r'
            )
            
        mean_self_loss = np.mean(self_loss_list)
        mean_self_acc = np.mean(self_acc_list)
        mean_exclusive_loss = np.mean(exclusive_loss_list)
        mean_exclusive_acc = np.mean(exclusive_acc_list)
        mean_f1_score = np.mean(f1_score_list)
                
        var_self_loss = np.var(self_loss_list)
        var_self_acc = np.var(self_acc_list)
        var_exclusive_loss = np.var(exclusive_loss_list)
        var_exclusive_acc = np.var(exclusive_acc_list)
        var_f1_score = np.var(f1_score_list)

        print(
        f"{mean_exclusive_acc:2.2f} $\pm$ {var_exclusive_acc:2.2f} & " +
        f"{mean_exclusive_loss:.4f} $\pm$ {var_exclusive_loss:.4f} & ", end=""
        )
        print(
        f"{mean_self_acc:2.2f} $\pm$ {var_self_acc:2.2f} & " +
        f"{mean_self_loss:.4f} $\pm$ {var_self_loss:.4f} "
        )


TopNActivations - ratio: 10.0%


NameError: name 'f1_score_list' is not defined